In [95]:
import firebase_admin
from firebase_admin import credentials, storage, firestore
#from google.colab import userdata
import json
import os

cred = credentials.Certificate("./serviceAccountKey.json")

if not firebase_admin._apps:
    firebase_admin.initialize_app(cred, {
        'storageBucket': 'fairplayfairy-3e2eb.firebasestorage.app'
    })
    print("Firebase 앱이 성공적으로 초기화되었습니다.")
else:
    print("Firebase 앱이 이미 초기화되어 있습니다.")
    
# Storage 버킷 객체 가져오기 예시
bucket = storage.bucket()
print("Storage 버킷에 접근 성공:", bucket.name)
# Database 객체 가져오기 예시
database = firestore.client()
print("Database에 접근 성공:", database)

Firebase 앱이 이미 초기화되어 있습니다.
Storage 버킷에 접근 성공: fairplayfairy-3e2eb.firebasestorage.app
Database에 접근 성공: <google.cloud.firestore_v1.client.Client object at 0x00000226FF7DB4D0>


In [ ]:
import cv2
import numpy as np
import tempfile
import os
# Jupyter Notebook 첫 번째 셀에서 'from firebase_admin import firestore'가 실행되어야 합니다.

def analyze_full_accuracy(video_blob):
    """
    영상 분석을 위한 최종 통합 함수.
    Firestore에서 메타데이터를 가져오고, Storage에서 영상을 다운로드하여
    킬, 헤드샷, 일반 명중, 발사 횟수를 분석하고 최종 명중률을 계산합니다.
    """
 
    #  Storage 영상을 임시 파일로 다운로드
    fd, temp_path = tempfile.mkstemp(suffix=".webm")
    os.close(fd)

    # Firestore에서 메타데이터 가져오기
    try:
        firestore_client = firestore.client()
        doc_id = video_blob.name.replace('videos/', '').replace('.webm', '')
        doc_ref = firestore_client.collection('game_results').document(doc_id)
        metadata_snapshot = doc_ref.get()
        if metadata_snapshot.exists:
            metadata = metadata_snapshot.to_dict()
            print("Firestore 메타데이터 가져오기 성공:", metadata)
        else:
            print(f"Firestore에서 문서를 찾을 수 없습니다: game_results/{doc_id}")
            return
    except Exception as e:
        print(f"Firestore에서 데이터를 가져오는 중 오류 발생: {e}")
        return

    fd, temp_path = tempfile.mkstemp(suffix=".webm")
    os.close(fd)

    try:
        print(f"영상을 임시 파일로 다운로드 중... -> {temp_path}")
        video_blob.download_to_filename(temp_path)
        print("다운로드 완료.")

        cap = cv2.VideoCapture(temp_path)
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        fps = cap.get(cv2.CAP_PROP_FPS) or 30.0

        # --- ROI: 크로스헤어 중심, 총구는 오른쪽-아래 오프셋 + 더 크게 ---
        crosshair_size = max(24, int(min(height, width) * 0.05))
        ch_y, ch_x = int(height/2 - crosshair_size/2), int(width/2 - crosshair_size/2)

        # 총구 화염 ROI 크기(크로스헤어보다 큼)
        muzzle_h = muzzle_w = max(120, int(min(height, width) * 0.14))
        # 오른쪽-아래 오프셋(해상도 비례) → 필요 시 수치 미세조정
        offset_y = int(height * 0.06)   # 아래쪽
        offset_x = int(width  * 0.08)   # 오른쪽
        mz_cy = int(height/2) + offset_y
        mz_cx = int(width/2)  + offset_x
        mz_y = max(0, mz_cy - muzzle_h // 2)
        mz_x = max(0, mz_cx - muzzle_w // 2)

        # --- 색상 범위 ---
        lower_red1, upper_red1 = np.array([0, 120, 120]),  np.array([10, 255, 255])
        lower_red2, upper_red2 = np.array([170, 120, 120]), np.array([180, 255, 255])
        lower_orange, upper_orange = np.array([8, 150, 160]), np.array([25, 255, 255])
        lower_white, upper_white = np.array([0, 0, 220]),   np.array([180, 40, 255])
        lower_yellow, upper_yellow = np.array([15, 120, 170]), np.array([35, 255, 255])

        # --- 상태/시간 파라미터(밀리초 기반) ---
        COOLDOWN_MS     = 100
        HIT_IGNORE_MS   = 90    # 발사 직후 잠깐 대기(총구 화염 간섭 회피)
        HIT_WINDOW_MS   = 240   # 이 안에서만 명중 판정
        COOLDOWN_FRAMES   = max(1, int(COOLDOWN_MS   * fps / 1000))
        HIT_IGNORE_FRAMES = max(1, int(HIT_IGNORE_MS * fps / 1000))
        HIT_SCAN_WINDOW   = max(HIT_IGNORE_FRAMES + 1, int(HIT_WINDOW_MS * fps / 1000))

        # 비율 임계값(ROI 대비 마스크 픽셀 비율)
        SHOT_RATIO_TH = 0.015   # 총구 노랑 비율 1.5%↑ → 발사
        HIT_RATIO_TH  = 0.010   # 크로스헤어 색 비율 1.0%↑ → 명중류

        headshot_count = hit_count = kill_count = shots_fired_count = 0
        shot_cooldown = 0
        hit_scan_state = {"active": False, "frames_left": 0}

        kernel = np.ones((3,3), np.uint8)

        print("종합 분석을 시작합니다 (오프셋/타이밍 보정)...")

        while True:
            ret, frame = cap.read()
            if not ret: break

            if shot_cooldown > 0: shot_cooldown -= 1
            if hit_scan_state["active"]: hit_scan_state["frames_left"] -= 1

            # --- 발사 감지(총구: 오른쪽-아래 큰 ROI, 비율 기반) ---
            if shot_cooldown == 0:
                muzzle_roi = frame[mz_y:min(height, mz_y + muzzle_h), mz_x:min(width, mz_x + muzzle_w)]
                if muzzle_roi.size > 0:
                    hsv_muzzle = cv2.cvtColor(muzzle_roi, cv2.COLOR_BGR2HSV)
                    yellow_mask = cv2.inRange(hsv_muzzle, lower_yellow, upper_yellow)
                    yellow_mask = cv2.dilate(yellow_mask, kernel, iterations=1)
                    muzzle_ratio = cv2.countNonZero(yellow_mask) / yellow_mask.size

                    if muzzle_ratio >= SHOT_RATIO_TH:
                        shots_fired_count += 1
                        shot_cooldown = COOLDOWN_FRAMES
                        hit_scan_state = {"active": True, "frames_left": HIT_SCAN_WINDOW}

            # --- 명중 감지(발사 후 지연(HIT_IGNORE) 이후에만) ---
            if hit_scan_state["active"] and hit_scan_state["frames_left"] < (HIT_SCAN_WINDOW - HIT_IGNORE_FRAMES):
                crosshair_roi = frame[ch_y:ch_y+crosshair_size, ch_x:ch_x+crosshair_size]
                hsv_crosshair = cv2.cvtColor(crosshair_roi, cv2.COLOR_BGR2HSV)

                red_mask = cv2.inRange(hsv_crosshair, lower_red1, upper_red1) + \
                           cv2.inRange(hsv_crosshair, lower_red2, upper_red2)
                orange_mask = cv2.inRange(hsv_crosshair, lower_orange, upper_orange)
                white_mask  = cv2.inRange(hsv_crosshair, lower_white, upper_white)

                red_mask    = cv2.morphologyEx(red_mask,    cv2.MORPH_OPEN, kernel, iterations=1)
                orange_mask = cv2.morphologyEx(orange_mask, cv2.MORPH_OPEN, kernel, iterations=1)
                white_mask  = cv2.morphologyEx(white_mask,  cv2.MORPH_OPEN, kernel, iterations=1)

                red_ratio    = cv2.countNonZero(red_mask)    / red_mask.size
                orange_ratio = cv2.countNonZero(orange_mask) / orange_mask.size
                white_ratio  = cv2.countNonZero(white_mask)  / white_mask.size

                if red_ratio >= HIT_RATIO_TH:
                    kill_count += 1
                    hit_scan_state["active"] = False
                elif orange_ratio >= HIT_RATIO_TH:
                    headshot_count += 1
                    hit_scan_state["active"] = False
                elif white_ratio >= HIT_RATIO_TH:
                    hit_count += 1
                    hit_scan_state["active"] = False

            # 명중 확인 시간이 모두 지나면 스캔 종료
            if hit_scan_state["frames_left"] <= 0:
                hit_scan_state["active"] = False

        # ... (이하 결과 출력 및 파일 정리 부분은 동일) ...
        cap.release()
        total_hits = kill_count + headshot_count + hit_count
        print("\n\n--- 최종 분석 결과 ---")
        print(f"총 발사 수: {shots_fired_count}발")
        print(f"총 명중 수: {total_hits}회")
        print(f"  - 킬: {kill_count}회")
        print(f"  - 헤드샷 (킬 제외): {headshot_count}회")
        print(f"  - 일반 명중 (킬, 헤드샷 제외): {hit_count}회")

        if shots_fired_count > 0:
            accuracy = (total_hits / shots_fired_count) * 100
            print(f"\n🎯 최종 명중률: {accuracy:.2f}%")
        else:
            print("\n🎯 발사 기록이 없어 명중률을 계산할 수 없습니다.")
    finally:
        print(f"임시 파일 삭제: {temp_path}")
        if os.path.exists(temp_path):
            os.remove(temp_path)

In [97]:
import random

if __name__ == "__main__":
    videos = list(bucket.list_blobs(prefix='videos/'))
    sample_videos = random.sample(videos, 10)
    for video_blob in sample_videos:
        analyze_full_accuracy(video_blob)

Firestore 메타데이터 가져오기 성공: {'finalScore': 15, 'movingHits': 13, 'totalHeadshots': 11, 'peekingHits': 16, 'accuracy': 41.43, 'totalHits': 29, 'totalShots': 70}
영상을 임시 파일로 다운로드 중... -> C:\Users\jiho\AppData\Local\Temp\tmpg5bzs_4r.webm
다운로드 완료.
종합 분석을 시작합니다 (최종 로직)...


--- 최종 분석 결과 ---
총 발사 수: 31발
총 명중 수: 17회
  - 킬: 3회
  - 헤드샷 (킬 제외): 7회
  - 일반 명중 (킬, 헤드샷 제외): 7회

🎯 최종 명중률: 54.84%
임시 파일 삭제: C:\Users\jiho\AppData\Local\Temp\tmpg5bzs_4r.webm
Firestore 메타데이터 가져오기 성공: {'finalScore': 30, 'movingHits': 13, 'totalHeadshots': 30, 'peekingHits': 20, 'accuracy': 55, 'totalHits': 33, 'totalShots': 60}
영상을 임시 파일로 다운로드 중... -> C:\Users\jiho\AppData\Local\Temp\tmp_tv9dupg.webm
다운로드 완료.
종합 분석을 시작합니다 (최종 로직)...


--- 최종 분석 결과 ---
총 발사 수: 25발
총 명중 수: 10회
  - 킬: 0회
  - 헤드샷 (킬 제외): 0회
  - 일반 명중 (킬, 헤드샷 제외): 10회

🎯 최종 명중률: 40.00%
임시 파일 삭제: C:\Users\jiho\AppData\Local\Temp\tmp_tv9dupg.webm
Firestore 메타데이터 가져오기 성공: {'finalScore': 11, 'movingHits': 10, 'totalHeadshots': 10, 'peekingHits': 11, 'accuracy': 33.33, 